# Resume Classification using GPT-3
## Part 1: Data Preprocessing

### Team 12

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_name = 'UpdatedResumeDataSet'
data_path = '/content/' + file_name + '.csv'

In [ ]:
df = pd.read_csv(data_path)
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


## Removing Stopwords and Lemmatizing

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

In [ ]:
def clean(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text = re.sub(r"http\S+", "",text)
    html=re.compile(r'<.*?>')
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'')
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    text = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(text)
    return text

In [ ]:
df['Resume'] = df['Resume'].apply(lambda x: clean(x))
df.head()

,Category,Resume
0,Data Science,"skill programming language python pandas, nump..."
1,Data Science,education detail may may uit rgpv data scienti...
2,Data Science,"area interest deep learning, control system de..."
3,Data Science,skill r python sap hana tableau sap hana sql s...
4,Data Science,"education detail mca ymcaust, faridabad, harya..."


## Find the 10 most frequent words, hand-pick the uninformative ones, and remove those words.

The purpose is to reduce the size of prompts that would be feed into GPT-3, which would save both time and money with little to none sacrifice on classification accuracy (might actually increase accuracy due to the increase in Signal-to-Noise Ratio, more expriments are needed for a conclusion).

In [ ]:
from collections import Counter

In [ ]:
Counter(" ".join(df["Resume"]).split()).most_common(20)

[('project', 3990),
 ('exprience', 3829),
 ('company', 3571),
 ('month', 3344),
 ('detail', 3132),
 ('description', 3122),
 ('data', 2086),
 ('team', 2074),
 ('skill', 2017),
 ('system', 1848),
 (',', 1845),
 ('management', 1811),
 ('year', 1505),
 ('database', 1469),
 ('client', 1460),
 ('maharashtra', 1376),
 ('application', 1325),
 ('service', 1295),
 ('technology', 1290),
 ('testing', 1266)]

In [ ]:
uninformative_words = ['project',
                       'experience',
                       'company',
                       'detail',
                       'description',
                       'skill',
                       ',',
                       'year',
                       ]
#I PERSONALLY think that these words are not strong predictors for resume classification
for word in uninformative_words:
    df['Resume'] = df['Resume'].str.replace(word, '')

In [ ]:
Counter(" ".join(df["Resume"]).split()).most_common(10)

[('exprience', 3829),
 ('month', 3344),
 ('data', 2156),
 ('team', 2137),
 ('management', 2024),
 ('system', 1922),
 ('database', 1526),
 ('client', 1463),
 ('maharashtra', 1449),
 ('testing', 1349)]

Now looks better.

## Saving to csv

In [ ]:
file_name = 'UpdatedResumeDataset_preprocessed'
data_path = '/content/' + file_name + '.csv'

In [ ]:
df.to_csv(data_path)

## End of Data Preprocessing